In [28]:
library(randomForest)
library(tictoc)
library('caret')
library(geepack)
library(randomForestSRC)

In [29]:
# install.packages("BBmisc")
library("BBmisc")

In [30]:
data = read.csv('../../AnnotateData/recgli_annot10_CELorNELorUNK.csv')

In [31]:
data = data[,c('vialid','t_number_y', 'CELorNEL',
               'newdata', 'nadc.1','nfa.1', 
               'cbvn_nlin', 'ccri', 'cni', 'ncho', 'nnaa', 'ncre', 'outcome')]



In [32]:
data = data[which(data$outcome == "txe" | data$outcome == "rHGG"),]

In [33]:
data = data[which(data$CELorNEL != "unk"),]

In [34]:
table(is.na(data$cni), is.na(data$cbvn_nlin))

       
        FALSE TRUE
  FALSE   119   37
  TRUE     76   60

In [35]:
colSums(is.na(data))

vialid t_number_y   CELorNEL    newdata     nadc.1      nfa.1  cbvn_nlin 
         0          0          0          0         15         15         97 
      ccri        cni       ncho       nnaa       ncre    outcome 
       136        136        136        136        136          0

In [36]:
data = droplevels.data.frame(data)

In [37]:
head(data)

,vialid,t_number_y,CELorNEL,newdata,nadc.1,nfa.1,cbvn_nlin,ccri,cni,ncho,nnaa,ncre,outcome
1,1B82,10366,NEL,1,0.75,0.83,1.08,1.88,2.60,1.03,0.46,0.47,txe
3,29B82,10448,CEL,1,1.62,0.50,2.43,NA,NA,NA,NA,NA,rHGG
5,60B90,10594,NEL,1,2.12,0.42,0.95,1.03,3.27,1.04,0.36,0.79,rHGG
6,58B90,10594,NEL,1,1.60,0.73,0.84,2.33,7.57,1.61,0.13,1.07,rHGG
7,59B90,10594,NEL,1,3.03,0.42,0.42,2.13,4.37,1.89,0.74,1.21,rHGG
8,7B91,10655,NEL,1,1.15,0.61,0.63,2.22,3.43,1.10,0.27,0.53,txe


In [38]:
data_new = data[data$newdata ==1, ]

In [39]:
data_old = data[data$newdata ==0, ]

In [40]:
## ensure normalization 
for (col in c('nadc.1','nfa.1', 'cbvn_nlin', 
              'ccri', 'cni', 'ncho', 'nnaa', 'ncre')){
    data[col] = normalize(data[col], method = "standardize")
}

In [41]:
head(data)

,vialid,t_number_y,CELorNEL,newdata,nadc.1,nfa.1,cbvn_nlin,ccri,cni,ncho,nnaa,ncre,outcome
1,1B82,10366,NEL,1,-1.9306504,0.0887003,-0.4167726,-0.15160317,-0.3249772,-0.2834404,0.26413101,-0.2329996,txe
3,29B82,10448,CEL,1,-0.1270785,-0.9415532,0.8249134,NA,NA,NA,NA,NA,rHGG
5,60B90,10594,NEL,1,0.9094571,-1.1913116,-0.5363423,-0.45804495,-0.1544364,-0.2727651,-0.06445302,0.2409752,rHGG
6,58B90,10594,NEL,1,-0.1685399,-0.2234977,-0.6375167,0.01063071,0.9400795,0.3357215,-0.82019628,0.6557031,rHGG
7,59B90,10594,NEL,1,2.7959518,-1.1913116,-1.0238190,-0.06147324,0.1255560,0.6346272,1.18416628,0.8630670,rHGG
8,7B91,10655,NEL,1,-1.1014220,-0.5981353,-0.8306679,-0.02902646,-0.1137102,-0.2087139,-0.36017865,-0.1441293,txe


In [42]:
data_dummyVars = data.frame(data, 
                           hasPerf = ifelse(is.na(data$cbvn_nlin), 0, 1), 
                           hasSpec = ifelse(is.na(data$cni), 0, 1), 
                           isCEL = ifelse(data$CELorNEL == "CEL", 1, 0))

In [43]:
data_noMissingValues = data_dummyVars[!is.na(data_dummyVars$cbvn_nlin) & !is.na(data_dummyVars$cni),]
dim(data_noMissingValues)

[1] 119  16

In [44]:
data_noMissingValues$txe_rhgg = ifelse(data_noMissingValues$outcome=="txe", 0, 1)


In [45]:
dim(data_noMissingValues)

[1] 119  17

In [46]:
gee.fit1 = geeglm(txe_rhgg ~ isCEL*(cbvn_nlin+cni+ncho+ccri), data = data_noMissingValues, id = factor(t_number_y), family = binomial, corstr = "exchangeable")
round(summary(gee.fit1)$coefficients, 4)



,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),1.2885,0.3241,15.8017,0.0001
isCEL,-0.0942,0.3110,0.0917,0.7620
cbvn_nlin,0.1222,0.2588,0.2232,0.6366
cni,0.8486,0.8225,1.0646,0.3022
ncho,0.5615,0.7458,0.5668,0.4515
ccri,-0.5391,0.6770,0.6341,0.4258
isCEL:cbvn_nlin,-0.0445,0.2910,0.0234,0.8785
isCEL:cni,-0.8499,0.9603,0.7833,0.3761
isCEL:ncho,-0.6374,0.8023,0.6312,0.4269
isCEL:ccri,0.6442,0.6169,1.0905,0.2964


In [47]:
gee.fit1 = geeglm(txe_rhgg ~ isCEL*(cni+ncho+ccri)+cbvn_nlin, data = data_noMissingValues, id = factor(t_number_y), family = binomial, corstr = "exchangeable")
round(summary(gee.fit1)$coefficients, 4)



,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),1.2670,0.3194,15.7374,0.0001
isCEL,-0.0740,0.3264,0.0514,0.8207
cni,0.8699,0.8138,1.1426,0.2851
ncho,0.5720,0.7234,0.6250,0.4292
ccri,-0.5663,0.5350,1.1204,0.2898
cbvn_nlin,0.0860,0.2135,0.1621,0.6872
isCEL:cni,-0.8793,0.9565,0.8452,0.3579
isCEL:ncho,-0.6374,0.8030,0.6302,0.4273
isCEL:ccri,0.6699,0.4997,1.7969,0.1801


In [48]:
gee.fit1 = geeglm(txe_rhgg ~ isCEL:cni+isCEL:ncho+isCEL:ccri+cni+ncho+ccri+cbvn_nlin, data = data_noMissingValues, id = factor(t_number_y), family = binomial, corstr = "exchangeable")
round(summary(gee.fit1)$coefficients, 4)



,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),1.2181,0.2845,18.3356,0.0000
cni,0.8475,0.8331,1.0348,0.3090
ncho,0.5501,0.6971,0.6226,0.4301
ccri,-0.5628,0.5168,1.1857,0.2762
cbvn_nlin,0.0650,0.1989,0.1067,0.7440
isCEL:cni,-0.8452,1.0017,0.7120,0.3988
isCEL:ncho,-0.6198,0.7889,0.6172,0.4321
isCEL:ccri,0.6559,0.4785,1.8791,0.1704


In [49]:
gee.fit1 = geeglm(txe_rhgg ~ isCEL:cni+isCEL:ncho+isCEL:ccri+cni+ncho+ccri, data = data_noMissingValues, id = factor(t_number_y), family = binomial, corstr = "exchangeable")
round(summary(gee.fit1)$coefficients, 4)



,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),1.2090,0.2872,17.7208,0.0000
cni,0.9114,0.8394,1.1790,0.2776
ncho,0.5308,0.7007,0.5739,0.4487
ccri,-0.5916,0.5255,1.2677,0.2602
isCEL:cni,-0.9176,0.9247,0.9849,0.3210
isCEL:ncho,-0.6175,0.7870,0.6156,0.4327
isCEL:ccri,0.7029,0.4745,2.1946,0.1385


In [50]:
gee.fit1 = geeglm(txe_rhgg ~ isCEL:cni+isCEL:ncho+isCEL:ccri+cni+ccri, data = data_noMissingValues, id = factor(t_number_y), family = binomial, corstr = "exchangeable")
round(summary(gee.fit1)$coefficients, 4)



,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),1.2289,0.2872,18.3070,0.0000
cni,1.2254,0.5762,4.5225,0.0335
ccri,-0.4106,0.4808,0.7293,0.3931
isCEL:cni,-1.2058,0.7627,2.4996,0.1139
isCEL:ncho,-0.1291,0.6210,0.0432,0.8354
isCEL:ccri,0.5249,0.4495,1.3640,0.2428


In [ ]:
gee.fit1 = geeglm(txe_rhgg ~ isCEL:cni+isCEL:ccri+cni+ccri, data = data_noMissingValues, id = factor(t_number_y), family = binomial, corstr = "exchangeable")
round(summary(gee.fit1)$coefficients, 4)



In [ ]:
gee.fit1 = geeglm(txe_rhgg ~ isCEL:cni+isCEL:ccri+cni, data = data_noMissingValues, id = factor(t_number_y), family = binomial, corstr = "exchangeable")
round(summary(gee.fit1)$coefficients, 4)



In [ ]:
gee.fit1 = geeglm(txe_rhgg ~ isCEL:cni+cni, data = data_noMissingValues, id = factor(t_number_y), family = binomial, corstr = "exchangeable")
round(summary(gee.fit1)$coefficients, 4)



In [75]:
gee.fit1 = geeglm(txe_rhgg ~ cni, data = data_noMissingValues, id = factor(t_number_y), family = binomial, corstr = "exchangeable")
round(summary(gee.fit1)$coefficients, 4)



,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),1.1963,0.2832,17.8413,0.0000
cni,0.2602,0.1988,1.7138,0.1905


## using all variables, imputing median for missing stuff: 

In [76]:
head(data)

,vialid,t_number_y,CELorNEL,newdata,nadc.1,nfa.1,cbvn_nlin,ccri,cni,ncho,nnaa,ncre,outcome
1,1B82,10366,NEL,1,-1.9306504,0.0887003,-0.4167726,-0.15160317,-0.3249772,-0.2834404,0.26413101,-0.2329996,txe
3,29B82,10448,CEL,1,-0.1270785,-0.9415532,0.8249134,NA,NA,NA,NA,NA,rHGG
5,60B90,10594,NEL,1,0.9094571,-1.1913116,-0.5363423,-0.45804495,-0.1544364,-0.2727651,-0.06445302,0.2409752,rHGG
6,58B90,10594,NEL,1,-0.1685399,-0.2234977,-0.6375167,0.01063071,0.9400795,0.3357215,-0.82019628,0.6557031,rHGG
7,59B90,10594,NEL,1,2.7959518,-1.1913116,-1.0238190,-0.06147324,0.1255560,0.6346272,1.18416628,0.8630670,rHGG
8,7B91,10655,NEL,1,-1.1014220,-0.5981353,-0.8306679,-0.02902646,-0.1137102,-0.2087139,-0.36017865,-0.1441293,txe


In [82]:
data_dummyVars = data.frame(data, 
                           hasPerf = ifelse(is.na(data$cbvn_nlin), 0, 1), 
                           hasSpec = ifelse(is.na(data$cni), 0, 1), 
                           isCEL = ifelse(data$CELorNEL == "CEL", 1, 0))

In [83]:
head(data_dummyVars)

,vialid,t_number_y,CELorNEL,newdata,nadc.1,nfa.1,cbvn_nlin,ccri,cni,ncho,nnaa,ncre,outcome,hasPerf,hasSpec,isCEL
1,1B82,10366,NEL,1,-1.9306504,0.0887003,-0.4167726,-0.15160317,-0.3249772,-0.2834404,0.26413101,-0.2329996,txe,1,1,0
3,29B82,10448,CEL,1,-0.1270785,-0.9415532,0.8249134,NA,NA,NA,NA,NA,rHGG,1,0,1
5,60B90,10594,NEL,1,0.9094571,-1.1913116,-0.5363423,-0.45804495,-0.1544364,-0.2727651,-0.06445302,0.2409752,rHGG,1,1,0
6,58B90,10594,NEL,1,-0.1685399,-0.2234977,-0.6375167,0.01063071,0.9400795,0.3357215,-0.82019628,0.6557031,rHGG,1,1,0
7,59B90,10594,NEL,1,2.7959518,-1.1913116,-1.0238190,-0.06147324,0.1255560,0.6346272,1.18416628,0.8630670,rHGG,1,1,0
8,7B91,10655,NEL,1,-1.1014220,-0.5981353,-0.8306679,-0.02902646,-0.1137102,-0.2087139,-0.36017865,-0.1441293,txe,1,1,0


In [88]:
data_dummyVars$cbvn_nlin = ifelse(is.na(data_dummyVars$cbvn_nlin), median(data_dummyVars$cbvn_nlin, na.rm = T), data_dummyVars$cbvn_nlin)
data_dummyVars$cni = ifelse(is.na(data_dummyVars$cni), median(data_dummyVars$cni, na.rm = T), data_dummyVars$cni)
data_dummyVars$ncho = ifelse(is.na(data_dummyVars$ncho), median(data_dummyVars$ncho, na.rm = T), data_dummyVars$ncho)
data_dummyVars$ccri = ifelse(is.na(data_dummyVars$ccri), median(data_dummyVars$ccri, na.rm = T), data_dummyVars$ccri)
data_dummyVars$nnaa = ifelse(is.na(data_dummyVars$nnaa), median(data_dummyVars$nnaa, na.rm = T), data_dummyVars$nnaa)
data_dummyVars$ncre = ifelse(is.na(data_dummyVars$ncre), median(data_dummyVars$ncre, na.rm = T), data_dummyVars$ncre)


data_dummyVars$nadc.1 = ifelse(is.na(data_dummyVars$nadc.1), median(data_dummyVars$nadc.1, na.rm = T), data_dummyVars$nadc.1)
data_dummyVars$nfa.1 = ifelse(is.na(data_dummyVars$nfa.1), median(data_dummyVars$nfa.1, na.rm = T), data_dummyVars$nfa.1)


In [89]:
colSums(is.na(data_dummyVars))

vialid t_number_y   CELorNEL    newdata     nadc.1      nfa.1  cbvn_nlin 
         0          0          0          0          0          0          0 
      ccri        cni       ncho       nnaa       ncre    outcome    hasPerf 
         0          0          0          0          0          0          0 
   hasSpec      isCEL 
         0          0

In [90]:
data_imp = data_dummyVars

In [94]:
dim(data_imp)

[1] 292  17

In [91]:
data_imp$txe_rhgg = ifelse(data_imp$outcome=="txe", 0, 1)


In [96]:
gee.fit1 = geeglm(txe_rhgg ~ isCEL*(cbvn_nlin+cni+ncho+ccri), data = data_imp, id = factor(t_number_y), family = binomial, corstr = "exchangeable")
round(summary(gee.fit1)$coefficients, 4)



,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),1.0175,0.2139,22.6378,0.0000
isCEL,-0.0599,0.2166,0.0764,0.7823
cbvn_nlin,0.0719,0.2817,0.0652,0.7985
cni,0.6237,0.6090,1.0489,0.3058
ncho,0.2646,0.5975,0.1961,0.6579
ccri,-0.1633,0.3103,0.2769,0.5988
isCEL:cbvn_nlin,0.3017,0.2653,1.2931,0.2555
isCEL:cni,-0.3518,0.7478,0.2213,0.6381
isCEL:ncho,-0.5798,0.7258,0.6381,0.4244
isCEL:ccri,0.2777,0.3848,0.5207,0.4705


In [97]:
gee.fit1 = geeglm(txe_rhgg ~ isCEL*(cni+ncho+ccri)+isCEL:cbvn_nlin, data = data_imp, id = factor(t_number_y), family = binomial, corstr = "exchangeable")
round(summary(gee.fit1)$coefficients, 4)



,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),0.9873,0.1995,24.4985,0.0000
isCEL,-0.0303,0.2124,0.0203,0.8866
cni,0.6503,0.6116,1.1308,0.2876
ncho,0.2597,0.6031,0.1855,0.6667
ccri,-0.1918,0.3031,0.4006,0.5268
isCEL:cni,-0.3934,0.7157,0.3022,0.5825
isCEL:ncho,-0.5603,0.7116,0.6199,0.4311
isCEL:ccri,0.3072,0.3802,0.6529,0.4191
isCEL:cbvn_nlin,0.3671,0.1784,4.2353,0.0396


In [98]:
gee.fit1 = geeglm(txe_rhgg ~ isCEL:(cni+ncho+ccri)+cni+ncho+ccri+isCEL:cbvn_nlin, data = data_imp, id = factor(t_number_y), family = binomial, corstr = "exchangeable")
round(summary(gee.fit1)$coefficients, 4)



,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),0.9678,0.1629,35.3096,0.0000
cni,0.6297,0.6234,1.0202,0.3125
ncho,0.2616,0.6005,0.1899,0.6630
ccri,-0.1979,0.2980,0.4410,0.5067
isCEL:cni,-0.3686,0.7180,0.2636,0.6077
isCEL:ncho,-0.5580,0.7095,0.6186,0.4316
isCEL:ccri,0.3074,0.3773,0.6639,0.4152
isCEL:cbvn_nlin,0.3684,0.1788,4.2434,0.0394


In [99]:
gee.fit1 = geeglm(txe_rhgg ~ isCEL:(cni+ncho+ccri)+cni+ccri+isCEL:cbvn_nlin, data = data_imp, id = factor(t_number_y), family = binomial, corstr = "exchangeable")
round(summary(gee.fit1)$coefficients, 4)



,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),0.9736,0.1610,36.5517,0.0000
cni,0.8298,0.3284,6.3853,0.0115
ccri,-0.1552,0.2737,0.3216,0.5706
isCEL:cni,-0.5516,0.5780,0.9106,0.3400
isCEL:ncho,-0.3160,0.5747,0.3024,0.5824
isCEL:ccri,0.2643,0.3638,0.5278,0.4675
isCEL:cbvn_nlin,0.3638,0.1780,4.1762,0.0410


In [100]:
gee.fit1 = geeglm(txe_rhgg ~ isCEL:(cni+ccri)+cni+ccri+isCEL:cbvn_nlin, data = data_imp, id = factor(t_number_y), family = binomial, corstr = "exchangeable")
round(summary(gee.fit1)$coefficients, 4)



,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),0.9812,0.1618,36.7683,0.0000
cni,0.8345,0.3274,6.4980,0.0108
ccri,-0.1446,0.2735,0.2794,0.5971
isCEL:cni,-0.7880,0.4324,3.3206,0.0684
isCEL:ccri,0.2017,0.3319,0.3694,0.5433
isCEL:cbvn_nlin,0.3716,0.1774,4.3860,0.0362


In [101]:
gee.fit1 = geeglm(txe_rhgg ~ isCEL:(cni)+cni+ccri+isCEL:cbvn_nlin, data = data_imp, id = factor(t_number_y), family = binomial, corstr = "exchangeable")
round(summary(gee.fit1)$coefficients, 4)



,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),0.9886,0.1617,37.3982,0.0000
cni,0.7635,0.2833,7.2646,0.0070
ccri,-0.0389,0.1970,0.0389,0.8436
isCEL:cni,-0.6292,0.3355,3.5173,0.0607
isCEL:cbvn_nlin,0.3715,0.1765,4.4296,0.0353


In [102]:
gee.fit1 = geeglm(txe_rhgg ~ isCEL:(cni)+cni+isCEL:cbvn_nlin, data = data_imp, id = factor(t_number_y), family = binomial, corstr = "exchangeable")
round(summary(gee.fit1)$coefficients, 4)



,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),0.9888,0.1616,37.4571,0.0000
cni,0.7332,0.2477,8.7611,0.0031
isCEL:cni,-0.6338,0.3305,3.6779,0.0551
isCEL:cbvn_nlin,0.3702,0.1764,4.4052,0.0358


## randomForestSRC imputation 

In [59]:
data_imp = read.csv('data_imputed.csv')

In [61]:
data_imp['isCEL'] = ifelse(data_imp$CELorNEL=="CEL", 1, 0)

In [64]:
data_imp['txe_rhgg'] = ifelse(data_imp$outcome=="rHGG", 1, 0)

In [65]:
head(data_imp)

X,vialid,t_number_y,CELorNEL,newdata,cbvn_nlin,recov_npar,ccri,cni,ncho,ncre,nnaa,nadc.1,nfa.1,outcome,isCEL,txe_rhgg
1,1B82,10366,NEL,1,1.08,98.89,1.880000,2.600000,1.030000,0.4700000,0.4600000,0.75,0.83,txe,0,0
3,29B82,10448,CEL,1,2.43,100.00,2.196667,3.685115,1.192184,0.5575862,0.3183908,1.62,0.50,rHGG,1,1
5,60B90,10594,NEL,1,0.95,100.00,1.030000,3.270000,1.040000,0.7900000,0.3600000,2.12,0.42,rHGG,0,1
6,58B90,10594,NEL,1,0.84,100.00,2.330000,7.570000,1.610000,1.0700000,0.1300000,1.60,0.73,rHGG,0,1
7,59B90,10594,NEL,1,0.42,100.00,2.130000,4.370000,1.890000,1.2100000,0.7400000,3.03,0.42,rHGG,0,1
8,7B91,10655,NEL,1,0.63,100.00,2.220000,3.430000,1.100000,0.5300000,0.2700000,1.15,0.61,txe,0,0


In [66]:
gee.fit1 = geeglm(txe_rhgg ~ isCEL*(cbvn_nlin+recov_npar+cni+ncho+ccri), data = data_imp, id = factor(t_number_y), family = binomial, corstr = "exchangeable")
round(summary(gee.fit1)$coefficients, 4)

,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),-1.2804,1.6828,0.5789,0.4467
isCEL,-0.5007,1.8113,0.0764,0.7822
cbvn_nlin,0.0706,0.2510,0.0790,0.7786
recov_npar,0.0152,0.0199,0.5863,0.4439
cni,0.0610,0.1511,0.1627,0.6867
ncho,0.4326,0.6437,0.4517,0.5015
ccri,-0.0448,0.0945,0.2248,0.6354
isCEL:cbvn_nlin,0.2866,0.2441,1.3790,0.2403
isCEL:recov_npar,0.0126,0.0222,0.3254,0.5684
isCEL:cni,0.0548,0.1961,0.0781,0.7799


In [67]:
gee.fit1 = geeglm(txe_rhgg ~ isCEL:(cbvn_nlin+recov_npar+cni+ncho+ccri)+cbvn_nlin+recov_npar+cni+ncho+ccri, data = data_imp, id = factor(t_number_y), family = binomial, corstr = "exchangeable")
round(summary(gee.fit1)$coefficients, 4)

,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),-1.6149,1.1536,1.9597,0.1616
cbvn_nlin,0.0755,0.2498,0.0914,0.7624
recov_npar,0.0191,0.0139,1.8986,0.1682
cni,0.0704,0.1500,0.2205,0.6387
ncho,0.3894,0.6293,0.3829,0.5361
ccri,-0.0439,0.0943,0.2166,0.6416
isCEL:cbvn_nlin,0.2691,0.2354,1.3070,0.2529
isCEL:recov_npar,0.0071,0.0076,0.8564,0.3548
isCEL:cni,0.0396,0.1867,0.0450,0.8321
isCEL:ncho,-1.0659,0.8400,1.6101,0.2045


In [70]:
gee.fit1 = geeglm(txe_rhgg ~ isCEL:(cbvn_nlin+recov_npar+ncho+ccri)+cbvn_nlin+recov_npar+cni+ncho+ccri, data = data_imp, id = factor(t_number_y), family = binomial, corstr = "exchangeable")
round(summary(gee.fit1)$coefficients, 4)

,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),-1.6233,1.1506,1.9904,0.1583
cbvn_nlin,0.0630,0.2376,0.0703,0.7909
recov_npar,0.0196,0.0135,2.1055,0.1468
cni,0.0905,0.1049,0.7445,0.3882
ncho,0.3239,0.5106,0.4025,0.5258
ccri,-0.0486,0.0946,0.2638,0.6075
isCEL:cbvn_nlin,0.2849,0.2275,1.5672,0.2106
isCEL:recov_npar,0.0063,0.0070,0.8270,0.3631
isCEL:ncho,-0.9297,0.4719,3.8815,0.0488
isCEL:ccri,0.1064,0.1382,0.5924,0.4415


In [71]:
gee.fit1 = geeglm(txe_rhgg ~ isCEL:(cbvn_nlin+recov_npar+ncho+ccri)+recov_npar+cni+ncho+ccri, data = data_imp, id = factor(t_number_y), family = binomial, corstr = "exchangeable")
round(summary(gee.fit1)$coefficients, 4)

,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),-1.5854,1.1458,1.9145,0.1665
recov_npar,0.0199,0.0133,2.2512,0.1335
cni,0.0955,0.1090,0.7688,0.3806
ncho,0.3285,0.5138,0.4088,0.5226
ccri,-0.0551,0.0965,0.3262,0.5679
isCEL:cbvn_nlin,0.3389,0.1456,5.4153,0.0200
isCEL:recov_npar,0.0058,0.0064,0.8246,0.3638
isCEL:ncho,-0.9491,0.4847,3.8344,0.0502
isCEL:ccri,0.1109,0.1392,0.6345,0.4257


In [72]:
gee.fit1 = geeglm(txe_rhgg ~ isCEL:(cbvn_nlin+recov_npar+ncho+ccri)+recov_npar+cni+ncho, data = data_imp, id = factor(t_number_y), family = binomial, corstr = "exchangeable")
round(summary(gee.fit1)$coefficients, 4)

,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),-1.6050,1.1423,1.9742,0.1600
recov_npar,0.0205,0.0131,2.4752,0.1157
cni,0.0861,0.1076,0.6391,0.4240
ncho,0.2427,0.4723,0.2641,0.6073
isCEL:cbvn_nlin,0.3383,0.1455,5.4069,0.0201
isCEL:recov_npar,0.0053,0.0063,0.7024,0.4020
isCEL:ncho,-0.8340,0.4358,3.6624,0.0557
isCEL:ccri,0.0602,0.1099,0.3008,0.5834


In [73]:
gee.fit1 = geeglm(txe_rhgg ~ isCEL:(cbvn_nlin+recov_npar+ncho+ccri)+recov_npar+cni, data = data_imp, id = factor(t_number_y), family = binomial, corstr = "exchangeable")
round(summary(gee.fit1)$coefficients, 4)

,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),-1.6460,1.1477,2.0570,0.1515
recov_npar,0.0227,0.0130,3.0522,0.0806
cni,0.1275,0.0507,6.3345,0.0118
isCEL:cbvn_nlin,0.3345,0.1458,5.2636,0.0218
isCEL:recov_npar,0.0043,0.0064,0.4604,0.4974
isCEL:ncho,-0.7408,0.4319,2.9419,0.0863
isCEL:ccri,0.0506,0.1100,0.2117,0.6454


In [74]:
gee.fit1 = geeglm(txe_rhgg ~ isCEL:(cbvn_nlin+recov_npar+ncho)+recov_npar+cni, data = data_imp, id = factor(t_number_y), family = binomial, corstr = "exchangeable")
round(summary(gee.fit1)$coefficients, 4)

,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),-1.6662,1.1422,2.1278,0.1446
recov_npar,0.0227,0.0130,3.0738,0.0796
cni,0.1313,0.0512,6.5802,0.0103
isCEL:cbvn_nlin,0.3368,0.1449,5.4046,0.0201
isCEL:recov_npar,0.0039,0.0063,0.3887,0.5330
isCEL:ncho,-0.6164,0.3101,3.9514,0.0468


In [75]:
gee.fit1 = geeglm(txe_rhgg ~ isCEL:(cbvn_nlin+ncho)+recov_npar+cni, data = data_imp, id = factor(t_number_y), family = binomial, corstr = "exchangeable")
round(summary(gee.fit1)$coefficients, 4)

,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),-1.7861,1.1126,2.5773,0.1084
recov_npar,0.0253,0.0121,4.3407,0.0372
cni,0.1076,0.0446,5.8216,0.0158
isCEL:cbvn_nlin,0.3958,0.1251,10.0182,0.0015
isCEL:ncho,-0.4415,0.1624,7.3950,0.0065
